# Find Raw Data from Folders of NWB files

### from a folder of NWB files like this:

```
nwb-files-folder/
    arbitrary-subdirectory-structure/
        2023_04_20-13-06-20.nwb
        2023_04_20-15-26-40.nwb
        2023_04_20-17-06-20.nwb
        ...
    folder2/
        2024_04_20-13-06-20.nwb
        ...
    ...
```

### It will generate a `reformat.py` script, that you can execute in your raw data folder:


```
raw/
    arbitrary-subdirectory-structure/
        TSeries-04202023-001/
        TSeries-04202023-002/
        TSeries-04202023-003/
        TSeries-04202023-004/
        TSeries-04202023-005/
        2023_04_20/13-06-20/
        2023_04_20/14-28-40/
        2023_04_20/15-26-40/
        2023_04_20/17-06-20/
        ...
    folder2/
        ...
    ...
```
as:
```
python reformat.py ./raw/ REFORMATED_FOLDER
```

### to copy all the raw data related to the NWB files in a folder `to-keep/`

```
REFORMATED_FOLDER/
        2023_04_20/
            13-06-20/
                TSeries-04202023-001/
            15-26-40/
                TSeries-04202023-003/
            17-06-20/
                TSeries-04202023-005/
        2023_04_20/
            ...
```

In [1]:
import sys, os
import numpy as np
sys.path.append(os.path.join(os.path.expanduser('~'), 'work', 'physion', 'src'))
import physion

import warnings
warnings.filterwarnings("ignore") # disable the UserWarning from pynwb (arrays are not well oriented)

ImportError: /usr/lib/x86_64-linux-gnu/libgssapi_krb5.so.2: undefined symbol: krb5_ser_context_init, version krb5_3_MIT

In [8]:

def find_folder_infos(datafolder, 
                      subfolder=None,
                      Nmax=1000,
                      verbose=True):

    DATASET = physion.analysis.read_NWB.scan_folder_for_NWBfiles(datafolder)
    
    if subfolder is not None:
        Files = np.sort([f for f in DATASET['files'] if (subfolder in f)])
    else:
        Files = np.sort(DATASET['files'])

    TSeries, DayFolders, TimeFolders = [], [], []
    for i, filename in enumerate(Files[:Nmax]):

        if verbose:
            print('- %s' % filename)
        data = physion.analysis.read_NWB.Data(filename, verbose=verbose)

        Description = str(data.nwbfile.processing['ophys'].description)
        # print('     * %s' % Description)
        TSeries.append('TSeries-'+Description.split('TSeries-')[2].split('/')[0])
        if verbose:
            print('     * TSeries-folder: %s' % TSeries[-1])
        DayFolders.append(str(data.nwbfile.identifier)[:10])
        TimeFolders.append(str(data.nwbfile.identifier)[11:])
        if verbose:
            print('     * Day-folder: %s' % DayFolders[-1])
            print('     * Time-folder: %s' % TimeFolders[-1])

    return TSeries, DayFolders, TimeFolders

def find_subfolder(folder_name, root_folder,
                   day_folder=None):
    if day_folder is not None:
        List = [f[0] for f in os.walk(root_folder)\
                if ((f[0].split(os.path.sep)[-1]==folder_name) and (day_folder in f[0]))]
    else:
        List = [f[0] for f in os.walk(root_folder)\
                        if f[0].split(os.path.sep)[-1]==folder_name]
    if len(List)>0:
        return List[0]
    else:
        return None
       


In [3]:
datafolder = os.path.join(os.path.expanduser('~') ,'CURATED', 'SST-WT-NR1-GluN3-2023')
root_folder = os.path.join(os.path.expanduser('~') , 'DATA', 'TADDY')
new_folder = 'to-keep'
# TSeries, DayFolders, TimeFolders = find_folder_infos(datafolder)
#script = build_bash_script(datafolder, new_folder, Nmax=1000, verbose=False)

In [4]:
# folder where we look for NWB files
datafolder = os.path.join(os.path.expanduser('~') ,'CURATED', 'SST-WT-NR1-GluN3-2023')
# find names of subfolders to look for:
TSeries, DayFolders, TimeFolders = find_folder_infos(datafolder, 
                                                     verbose=False)

inspecting the folder "/home/yann.zerlaut/CURATED/SST-WT-NR1-GluN3-2023" [...]
 -> found n=127 datafiles (in 152.7s) 


NameError: name 'np' is not defined

In [9]:
# folder where we look for NWB files
datafolder = os.path.join(os.path.expanduser('~') , 'DATA', 'TADDY')
# find names of subfolders to look for:
TSeries, DayFolders, TimeFolders = find_folder_infos(datafolder, 
                                                     subfolder='Assembled',
                                                     verbose=False)

inspecting the folder "/home/yann.zerlaut/DATA/TADDY" [...]
 -> found n=507 datafiles (in 527.4s) 


In [ ]:
script = """
import os, sys, shutil

root_folder = "/volume1/Taddy/GluN3_V1_InVivo_Imaging/processed/"
root_folder2 = "/volume1/Taddy/GluN3_V1_InVivo_Imaging/raw/"

target_folder = "/volume1/Taddy/SST-WT-Glun1KO-GluN3KO-2023"
if not os.path.isdir(target_folder):
    pass
    # os.mkdir(target_folder)

def find_subfolder(folder_name, root_folder,
                   day_folder=None):
    if day_folder is not None:
        List = [f[0] for f in os.walk(root_folder)\
                if (f[0].split(os.path.sep)[-1]==folder_name) and (day_folder in f[0])]
    else:
        List = [f[0] for f in os.walk(root_folder)\
                        if f[0].split(os.path.sep)[-1]==folder_name]
    if len(List)>0:
        return List[0]
    else:
        return None
        
"""

n=0
for i, d, t in zip(TSeries, DayFolders, TimeFolders):
    n+=1
    script += """
print('- %i) : %s %s')
""" % (n, d, t)
    script += """
imaging = find_subfolder("%s", root_folder)
if imaging is None:
    imaging = find_subfolder("%s", root_folder2)
if imaging is None:
    print(' !! NOT FOUND ', '%s')
else:
    print(imaging)
""" % (i, i, i)
    script += """
timeF = find_subfolder("%s", root_folder, day_folder="%s")
print(timeF)
print('')
""" % (t, d)
#print(script)

In [ ]:
exec(script)

In [6]:
# write as a bash script
script_name = 'rebuild.py'
with open(script_name, 'w') as f:
    f.write(script)

NameError: name 'script' is not defined

In [5]:
cat rebuild.py

cat: rebuild.py: No such file or directory
